In [4]:
"""
ARIMA + SARIMA + Prophet с Optuna
"""
import warnings
import numpy as np
import optuna
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from prophet import Prophet
from sklearn.metrics import mean_absolute_error
warnings.filterwarnings("ignore")

# Загрузка и подготовка данных
df = pd.read_excel("test.xlsx")
df["date"] = pd.to_datetime(df["date"], dayfirst=True, errors="coerce")
df["meaning"] = pd.to_numeric(df["meaning"], errors="coerce")
df = (
    df.dropna(subset=["date", "meaning"])
      .sort_values("date")
      .set_index("date")
)

series = df["meaning"]
# Для Prophet делаем DataFrame с колонками ds / y
data_prophet = series.reset_index()
data_prophet.columns = ["ds", "y"]

# Train / Test 
FORECAST_HORIZON = 12 # общий горизонт прогноза
test_steps = min(FORECAST_HORIZON, max(1, len(series) // 4))

train = series.iloc[:-test_steps]
test = series.iloc[-test_steps:]

train_p = data_prophet.iloc[:-test_steps].copy()
test_p = data_prophet.iloc[-test_steps:].copy()

# Функция метрики
def mape(y_true, y_pred) -> float:
    """MAPE"""
    y_true = np.array(y_true, dtype=float)
    y_pred = np.array(y_pred, dtype=float)
    eps = 1e-9
    denom = np.clip(np.abs(y_true), eps, None)
    return float(np.mean(np.abs((y_true - y_pred) / denom)) * 100)


# ARIMA + Optuna
def objective_arima(trial):
    p = trial.suggest_int("p", 0, 6)
    d = trial.suggest_int("d", 0, 3)
    q = trial.suggest_int("q", 0, 6)

    try:
        model = ARIMA(
            train,
            order=(p, d, q),
            enforce_stationarity=False,
            enforce_invertibility=False,
        )
        res = model.fit()
        fc = res.get_forecast(steps=len(test)).predicted_mean
        return mape(test, fc)
    except Exception:
        return 1e9

study_arima = optuna.create_study(direction="minimize")
study_arima.optimize(objective_arima, n_trials=40, show_progress_bar=True)

best_arima_params = study_arima.best_params
print(f"Лучшие параметры ARIMA: {best_arima_params}")

# финальная ARIMA на всём ряду
p_a, d_a, q_a = best_arima_params["p"], best_arima_params["d"], best_arima_params["q"]
arima_final = ARIMA(
    series,
    order=(p_a, d_a, q_a),
    enforce_stationarity=False,
    enforce_invertibility=False,
).fit()

# прогноз ARIMA на тест и метрики
arima_fc_test = arima_final.get_forecast(steps=test_steps).predicted_mean
arima_mae = mean_absolute_error(test, arima_fc_test)
arima_mape = mape(test, arima_fc_test)


# SARIMA + Optuna
SEASONAL_PERIOD = 12  #данные месячные

def objective_sarima(trial):
    p = trial.suggest_int("p", 0, 3)
    d = trial.suggest_int("d", 0, 2)
    q = trial.suggest_int("q", 0, 3)

    P = trial.suggest_int("P", 0, 2)
    D = trial.suggest_int("D", 0, 1)
    Q = trial.suggest_int("Q", 0, 2)

    try:
        model = SARIMAX(
            train,
            order=(p, d, q),
            seasonal_order=(P, D, Q, SEASONAL_PERIOD),
            enforce_stationarity=False,
            enforce_invertibility=False,
        )
        res = model.fit(disp=False)
        fc = res.get_forecast(steps=len(test)).predicted_mean
        return mape(test, fc)
    except Exception:
        return 1e9


print("\n=== SARIMA + Optuna ===")
study_sarima = optuna.create_study(direction="minimize")
study_sarima.optimize(objective_sarima, n_trials=40, show_progress_bar=True)

best_sarima_params = study_sarima.best_params
print(f"Лучшие параметры SARIMA: {best_sarima_params}")

p_s = best_sarima_params["p"]
d_s = best_sarima_params["d"]
q_s = best_sarima_params["q"]
P_s = best_sarima_params["P"]
D_s = best_sarima_params["D"]
Q_s = best_sarima_params["Q"]

sarima_final = SARIMAX(
    series,
    order=(p_s, d_s, q_s),
    seasonal_order=(P_s, D_s, Q_s, SEASONAL_PERIOD),
    enforce_stationarity=False,
    enforce_invertibility=False,
).fit(disp=False)

sarima_fc_test = sarima_final.get_forecast(steps=test_steps).predicted_mean
sarima_mae = mean_absolute_error(test, sarima_fc_test)
sarima_mape = mape(test, sarima_fc_test)



# Prophet + Optuna

def objective_prophet(trial):
    params = {
        "seasonality_mode": trial.suggest_categorical(
            "seasonality_mode", ["additive", "multiplicative"]
        ),
        "changepoint_prior_scale": trial.suggest_float(
            "changepoint_prior_scale", 0.01, 1.0, log=True
        ),
        "seasonality_prior_scale": trial.suggest_float(
            "seasonality_prior_scale", 0.1, 10.0, log=True
        ),
        "yearly_seasonality": trial.suggest_int(
            "yearly_seasonality", 5, 15
        ),
    }

    try:
        m = Prophet(
            growth="linear",
            weekly_seasonality=False,
            daily_seasonality=False,
            **params,
        )
        m.fit(train_p)

        future = m.make_future_dataframe(periods=test_steps, freq="MS")
        fcst = m.predict(future).tail(test_steps)

        return mape(test_p["y"], fcst["yhat"])
    except Exception:
        return 1e9


print("\n=== Prophet + Optuna ===")
study_prophet = optuna.create_study(direction="minimize")
study_prophet.optimize(objective_prophet, n_trials=30, show_progress_bar=True)

best_prophet_params = study_prophet.best_params
print(f"Лучшие параметры Prophet: {best_prophet_params}")

# финальный Prophet на всём ряду
m_best = Prophet(
    growth="linear",
    weekly_seasonality=False,
    daily_seasonality=False,
    **best_prophet_params,
)
m_best.fit(data_prophet)

future_test_p = m_best.make_future_dataframe(periods=test_steps, freq="MS")
fcst_test_p = m_best.predict(future_test_p).tail(test_steps)

prophet_mae = mean_absolute_error(test_p["y"], fcst_test_p["yhat"])
prophet_mape = mape(test_p["y"], fcst_test_p["yhat"])


# ============================
# Таблица метрик и выбор лучшей модели
# ============================

metrics = pd.DataFrame(
    {
        "MAE": [
            arima_mae,
            sarima_mae,
            prophet_mae,
        ],
        "MAPE": [
            arima_mape,
            sarima_mape,
            prophet_mape,
        ],
    },
    index=[
        "ARIMA + Optuna",
        "SARIMA + Optuna",
        "Prophet + Optuna",
    ],
)

print("\n=== Итоговая таблица метрик (test) ===")
print(metrics.round(4))

# критерий выбора лучшей модели:
# сначала минимальный MAPE, при равенстве — минимальный MAE
best_idx = metrics["MAPE"].idxmin()
candidates = metrics[metrics["MAPE"] == metrics.loc[best_idx, "MAPE"]]
if len(candidates) > 1:
    best_idx = candidates["MAE"].idxmin()

print(f"\nЛучшая модель по MAPE/MAE: {best_idx}")

# ============================
# Прогноз лучшей модели на FORECAST_HORIZON
# ============================

best_forecast = None

if best_idx == "ARIMA + Optuna":
    best_forecast = arima_final.get_forecast(steps=FORECAST_HORIZON).predicted_mean

elif best_idx == "SARIMA + Optuna":
    best_forecast = sarima_final.get_forecast(steps=FORECAST_HORIZON).predicted_mean

elif best_idx == "Prophet + Optuna":
    future_full_p = m_best.make_future_dataframe(periods=FORECAST_HORIZON, freq="MS")
    fcst_full_p = m_best.predict(future_full_p).tail(FORECAST_HORIZON)
    best_forecast = fcst_full_p.set_index("ds")["yhat"]

print(f"\nПрогноз лучшей модели ({best_idx}) на {FORECAST_HORIZON} периодов:")
print(best_forecast.round(4))


[I 2025-12-02 22:15:37,713] A new study created in memory with name: no-name-77c307b1-f505-4fa7-9335-5848d9bbcea6


Всего наблюдений: 36; train: 27, test: 9

=== ARIMA + Optuna ===


  0%|          | 0/40 [00:00<?, ?it/s]

[I 2025-12-02 22:15:37,761] Trial 0 finished with value: 13.49842569089712 and parameters: {'p': 1, 'd': 0, 'q': 1}. Best is trial 0 with value: 13.49842569089712.
[I 2025-12-02 22:15:37,824] Trial 1 finished with value: 0.0 and parameters: {'p': 4, 'd': 2, 'q': 4}. Best is trial 1 with value: 0.0.
[I 2025-12-02 22:15:38,002] Trial 2 finished with value: 0.0 and parameters: {'p': 4, 'd': 3, 'q': 6}. Best is trial 1 with value: 0.0.
[I 2025-12-02 22:15:38,108] Trial 3 finished with value: 7.319472723072247e-14 and parameters: {'p': 2, 'd': 3, 'q': 3}. Best is trial 1 with value: 0.0.
[I 2025-12-02 22:15:38,182] Trial 4 finished with value: 0.0 and parameters: {'p': 5, 'd': 3, 'q': 2}. Best is trial 1 with value: 0.0.
[I 2025-12-02 22:15:38,281] Trial 5 finished with value: 53005.3037422154 and parameters: {'p': 0, 'd': 0, 'q': 5}. Best is trial 1 with value: 0.0.
[I 2025-12-02 22:15:38,498] Trial 6 finished with value: 0.8147372127929295 and parameters: {'p': 1, 'd': 0, 'q': 6}. Best is

[I 2025-12-02 22:15:42,969] A new study created in memory with name: no-name-492f8dcf-974d-4eca-af0b-99d504386105


...

[I 2025-12-02 22:20:14,838] Trial 28 finished with value: 0.9201482024922084 and parameters: {'seasonality_mode': 'additive', 'changepoint_prior_scale': 0.04151535571798378, 'seasonality_prior_scale': 0.8365647714007288, 'yearly_seasonality': 5}. Best is trial 23 with value: 0.01136256687517836.


22:20:26 - cmdstanpy - INFO - Chain [1] done processing
22:20:26 - cmdstanpy - INFO - Chain [1] start processing


[I 2025-12-02 22:20:26,650] Trial 29 finished with value: 0.2920235878250435 and parameters: {'seasonality_mode': 'multiplicative', 'changepoint_prior_scale': 0.11164614522584058, 'seasonality_prior_scale': 3.127255795573873, 'yearly_seasonality': 9}. Best is trial 23 with value: 0.01136256687517836.
Лучшие параметры Prophet: {'seasonality_mode': 'additive', 'changepoint_prior_scale': 0.02584141784559122, 'seasonality_prior_scale': 2.7115311939997797, 'yearly_seasonality': 6}


22:20:38 - cmdstanpy - INFO - Chain [1] done processing



=== Итоговая таблица метрик (test) ===
                     MAE     MAPE
ARIMA + Optuna    9.0000  28.3102
SARIMA + Optuna   9.0000  28.3102
Prophet + Optuna  8.7202  27.4332

Лучшая модель по MAPE/MAE: Prophet + Optuna

Прогноз лучшей модели (Prophet + Optuna) на 12 периодов:
ds
2026-01-01    36.7635
2026-02-01    37.7395
2026-03-01    38.7029
2026-04-01    39.7122
2026-05-01    40.7130
2026-06-01    41.7124
2026-07-01    42.7128
2026-08-01    43.7127
2026-09-01    44.7127
2026-10-01    45.7126
2026-11-01    46.7125
2026-12-01    47.7133
Name: yhat, dtype: float64
